Write a program to estimate a depth map from a pair of calibrated images in Figure 4 using an MRF and graphcuts. The camera matrices are available here. Note that, for finding the epipolar lines using the provided camera matrices, you might want to use the following equation:

### camera metrices:
1221.2270770	0.0000000	479.5000000	
0.0000000	1221.2270770	269.5000000	
0.0000000	0.0000000	1.0000000	
1.0000000000	0.0000000000	0.0000000000	
0.0000000000	1.0000000000	0.0000000000	
0.0000000000	0.0000000000	1.0000000000	
0.0000000000	0.0000000000	0.0000000000	


1221.2270770	0.0000000	479.5000000	
0.0000000	1221.2270770	269.5000000	
0.0000000	0.0000000	1.0000000	
0.9998813487	0.0148994942	0.0039106989	
-0.0148907594	0.9998865876	-0.0022532664	
-0.0039438279	0.0021947658	0.9999898146	
-9.9909793759	0.2451742154	0.1650832670	

In [1]:
import numpy
import cv2
from gco import pygco
import matplotlib.image
import matplotlib.pyplot
import typing
from IPython.display import Image

/Users/admin/EE5371/CA2/.venv/lib/python3.10/site-packages/gco/__init__.py:7: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, name):


In [2]:
# @cache
def compute_distance(point1: numpy.ndarray, point2: numpy.ndarray) -> float:
    """
    compute distance of two pixels
    Parameters
    ----------
    point1: ndarray, uint8, shape=(3)
        point one
    point2: ndarray, uint8, shape=(3)
        point two
    """
    # return (abs(point1[0]-point2[0]) + abs(point1[1] - point2[1]) + abs(point1[2] - point2[2]) ) / 3 / 255
    # if not dividing 255 as the C pseudocode example, there will be a overflowing of MAX_VALUE of Integer when calling pycgo.cut_grid_graph
    return numpy.sum(numpy.abs(point1-point2)) /3 / 255

In [3]:
K1: numpy.mat = numpy.mat(data=[[1221.2270770,0,479.5],[0,1221.2270770,269.5],[0,0,1]])
R1: numpy.mat = numpy.mat(data=[[1,0,0],[0,1,0],[0,0,1]])
T1: numpy.mat = numpy.mat(data=[0,0,0]).T
K2: numpy.mat = numpy.mat(data=[[1221.2270770,0,479.5],[0,1221.227077,269.5],[0,0,1]])
R2: numpy.mat = numpy.mat(data=[[0.9998813487,0.0148994942,0.0039106989],[-0.0148907594,0.9998865876,-0.0022532664],[-0.0039438279,0.0021947658,0.9999898146]])
T2: numpy.mat = numpy.mat(data=[-9.9909793759,0.2451742154,0.1650832670]).T
K1_inv: numpy.mat =  numpy.mat(data=numpy.linalg.inv(K1))

point_part_matrice = K2 @ R2 @ R1.T @ K1_inv
epipolar_line_matrice = K2 @ R2.T @ (T1-T2)

img1: numpy.ndarray = cv2.imread("data/not_rectified_image_1.jpeg", cv2.IMREAD_COLOR)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
height1, width1, _ = img1.shape

img2: numpy.ndarray = cv2.imread("data/not_rectified_image_2.jpeg", cv2.IMREAD_COLOR)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img2 = cv2.resize(img2, (width1, height1))

height2, width2, _ = img2.shape

In [4]:
POINT_COUNT_THRETHOD = 0.9
def compute_approximate_disparity_max(img1: numpy.ndarray, img2: numpy.ndarray) -> float:
    """
    approximate according to the number of count pixels after projection
    based on SIFT to find the overlapping rate
    """
    total_pixel = img1.shape[0] * img1.shape[1]
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT().create()
    keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)
    index_params = dict(algorithm=0, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.7 * n.distance:
            good_matches.append(m)

    points1 = numpy.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    points2 = numpy.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    F, mask = cv2.findFundamentalMat(points1, points2, cv2.FM_RANSAC)

    overlap_ratio = numpy.sum(mask) / len(mask)
    target_count = overlap_ratio * POINT_COUNT_THRETHOD * total_pixel
    d_max = 10
    d_min = 0
    cur_count = 0
    loop_count = 0
    cur_height1, cur_width1, _ = img1.shape
    cur_height2, cur_width2, _ = img2.shape
    while cur_count < target_count or loop_count < 50:
        print(f"d_max:{d_max}")
        print(f"loop_count:{loop_count}")

        loop_count += 1
        cur_count = 0
        epipolar_line = d_max * epipolar_line_matrice
        for y1 in range(cur_height1):
            for x1 in range(cur_width1):
                xh1 = numpy.mat(data=[y1,x1, 1])
                xh2 = (point_part_matrice @ xh1.T) + epipolar_line
                y2, x2, scale = xh2
                y2 = int(y2/scale)
                x2 = int(x2/scale)
                if 0 <= y2 < cur_height2 and 0 <= x2 < cur_width2:
                    cur_count += 1
        print(f"cur_count:{cur_count}, target_count:{target_count}")
        if cur_count < target_count:
            gap = (d_max - d_min)
            d_max -= gap/2
        else:
            middle = (d_max + d_min) /2
            d_min = d_max
            d_max += middle
    return d_max
# d_max = compute_approximate_disparity_max(img1=img1, img2=img2)
d_max = 0.0068307541828973015

In [5]:
# img1.reshape(1,-1,3)
# STEPS:typing.List[int] = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    # part1 = K2.dot(R2.T).dot(R1).dot(invK1).dot(Xh)
# STEPS:typing.List[int] = [3,4,5,6,7,8,9,10,11,12]
STEPS:typing.List[int] = [20,30,40,6,7,8,9,10,11,12]


print(f"point_part_matrice:{point_part_matrice}")
tem_cost = []
unary_cost_dict = {}
for STEP in STEPS:
    depth_range = (d_max) / STEP
    unary_cost = numpy.ndarray = numpy.zeros(shape=(height1, width1, STEP), dtype=numpy.float64)
    for s in range(STEP):
        d = s * depth_range
        epipolar_line = d * epipolar_line_matrice
        pre_epipolar_line = (d-depth_range) * epipolar_line_matrice
        for y1 in range(height1):
            for x1 in range(width1):
                xh1 = numpy.mat(data=[y1,x1, 1])
                xh2 = (point_part_matrice @ xh1.T) + epipolar_line
                y21, x21, scale = xh2
                y21 = int(y21/scale)
                x21 = int(x21/scale)
                xh2 = (point_part_matrice @ xh1.T) + epipolar_line
                xh2 = (point_part_matrice @ xh1.T) + pre_epipolar_line
                y22, x22, scale = xh2
                y22 = int(y22/scale)
                x22 = int(x22/scale)
                tem_cost.clear()
                # print(f"x21:{x21}, x22:{x22}, y21:{y21}, y22:{y22}")
                min_x = min(x21, x22)
                min_y = min(y21,y22)
                max_x = max(x21,x22)
                max_y = min(y21,y22)
                for tem_x in range(max(min_x, 0), min(max_x + 1, width2)):
                    for tem_y in range(max(min_y, 0), min(max_y + 1, height2)):
                        tem_cost.append(compute_distance(point1=img1[y1,x1], point2=img2[tem_y, tem_x]))
                if len(tem_cost) == 0:
                    unary_cost[y1, x1, s] = 1
                else:
                    unary_cost[y1, x1, s] = min(tem_cost)
                        
                # if y21 < 0 or y21 >= height2 or x21 < 0 or x21 >= width2:
                #     unary_cost[y1, x1, s] = 1
                # else:
                #     unary_cost[y1, x1, s] = compute_distance(point1=img1[y1,x1], point2=img2[y2, x2])
    print(f"unary_cost:{unary_cost}")
    unary_cost_dict[STEP] = unary_cost


point_part_matrice:[[ 9.98332853e-01  1.57612424e-02  1.32270989e+00]
 [-1.57610821e-02  1.00037093e+00  4.70297891e+00]
 [-3.22939769e-06  1.79718075e-06  1.00105397e+00]]


unary_cost:[[[3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 3.20261438e-01
   3.46405229e-01 2.79738562e-01]
  [3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 5.34640523e-01
   3.07189542e-01 3.90849673e-01]
  [3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 5.79084967e-01
   4.91503268e-01 6.30065359e-01]
  ...
  [1.00653595e-01 1.64705882e-01 3.63398693e-01 ... 6.60130719e-01
   3.26797386e-01 3.26797386e-01]
  [3.00000000e+02 3.00000000e+02 3.00000000e+02 ... 3.00000000e+02
   3.00000000e+02 3.00000000e+02]
  [3.00000000e+02 3.00000000e+02 3.00000000e+02 ... 3.00000000e+02
   3.00000000e+02 3.00000000e+02]]

 [[3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 1.29411765e-01
   5.62091503e-01 3.79084967e-01]
  [3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 2.41830065e-01
   5.67320261e-01 4.49673203e-01]
  [3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 3.04575163e-01
   6.37908497e-01 4.18300654e-01]
  ...
  [1.00653595e-01 5.04575163e-01 3.67320261e-01 ... 6.60130719e-01

In [6]:
for STEP in STEPS:
    # LAMBDA = 0.01
    # LAMBDAS = [0.005, 0.006, 0.007, 0.01,0.02,0.03, 0.04, 0.05, 0.06,0.07,0.08,0.09,0.10]
    # LAMBDAS = [0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7, 0.8,0.9,0.10]
    # LAMBDAS = [0.01,0.02,0.04,0.08,0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7, 0.8,0.9,0.10]
    LAMBDAS = [0.001,0.002,0.004,0.008,0.01, 0.02, 0.03, 0.04,0.05, 0.06,0.07,0.08,0.09,0.1,0.11, 0.12,0.13,0.14]
    # LAMBDAS = [0.001,0.002,0.004,0.008,0.01, 0.02, 0.03, 0.04,0.05]
    # LAMBDAS = [0.001, 0.002,0.003,0.]

    i = numpy.arange(STEP, dtype=numpy.int32)
    j = numpy.arange(STEP, dtype=numpy.int32)
    fp = numpy.abs(i[: , numpy.newaxis] - j)
    # fp = (1 - numpy.eye(STEP,dtype=numpy.int32))
    unary_cost = unary_cost_dict[STEP]
    print(f"unary_cost:{unary_cost}")
    # fp = numpy.abs(i[: , numpy.newaxis] - j)
    for lam in LAMBDAS:
        # lam/=10
        labels = pygco.cut_grid_graph_simple(unary_cost=unary_cost, pairwise_cost=fp * lam, connect=8, algorithm="swap")
        new_min = 0
        new_max = 255

        min_value = numpy.min(labels)
        max_value = numpy.max(labels)
        print(f"labels:{labels}")
        labels = ((labels - min_value) / (max_value - min_value)) * (new_max - new_min) + new_min
        labels = labels.reshape(unary_cost.shape[0], unary_cost.shape[1])
        print(f"labels:{labels}")
        img_path = f"data/no-rectified-labels{STEP}-{lam}.png"
        cv2.imwrite(img_path, labels)
        Image(filename=img_path)
        # img_show = matplotlib.image.imread(img_path)
        # matplotlib.pyplot.title(img_path)
        # matplotlib.pyplot.imshow(X=labels, cmap='gray') 
        # matplotlib.pyplot.show()
        # lam += 0.01

unary_cost:[[[3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 3.20261438e-01
   3.46405229e-01 2.79738562e-01]
  [3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 5.34640523e-01
   3.07189542e-01 3.90849673e-01]
  [3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 5.79084967e-01
   4.91503268e-01 6.30065359e-01]
  ...
  [1.00653595e-01 1.64705882e-01 3.63398693e-01 ... 6.60130719e-01
   3.26797386e-01 3.26797386e-01]
  [3.00000000e+02 3.00000000e+02 3.00000000e+02 ... 3.00000000e+02
   3.00000000e+02 3.00000000e+02]
  [3.00000000e+02 3.00000000e+02 3.00000000e+02 ... 3.00000000e+02
   3.00000000e+02 3.00000000e+02]]

 [[3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 1.29411765e-01
   5.62091503e-01 3.79084967e-01]
  [3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 2.41830065e-01
   5.67320261e-01 4.49673203e-01]
  [3.00000000e+02 2.61437908e-03 2.61437908e-03 ... 3.04575163e-01
   6.37908497e-01 4.18300654e-01]
  ...
  [1.00653595e-01 5.04575163e-01 3.67320261e-01 ... 6.60130719e-01